# London Bicycles Data Analysis

## Project Overview

This project presents an analytical deep dive into the London Santander Cycle Hire Scheme, using the public dataset hosted on Google BigQuery. The dataset captures bike rental activity from 2011 onwards, offering rich insights into cycling behavior, station popularity, trip durations, and temporal trends across the city.

## Tools & Technologies

Google BigQuery / MySQL: For querying large datasets efficiently
Python: Data handling and analysis
Pandas & NumPy: Data manipulation and aggregation
Matplotlib & Seaborn: Data visualization
Google Cloud SDK: Accessing BigQuery data through Python

## Dataset Summary

The dataset contains millions of records, with key features including:

Start and End Timestamps of each ride
Start and End Station Names and IDs
Bike ID
Ride Duration (in seconds/minutes)
Station Metadata (location and identifiers)

## Key Questions Explored

What are the longest bike trips recorded by year?
How does daily ridership trend over time?
What were the most popular stations for bike rentals?
On specific dates (e.g., summer solstice), where did most bikes travel?
Which stations had the longest average ride durations?

## Goal

To uncover usage patterns, station performance, and temporal insights that can be useful for:

Urban planning
Infrastructure development
Understanding user behavior

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *

In [20]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "london_bicycle" dataset
dataset_ref = client.dataset("london_bicycles", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "cycle_hire" and "cycle_stations" tables
cycle_hire_table_ref = dataset_ref.table("cycle_hire")
cycle_stations_table_ref = dataset_ref.table("cycle_stations")

# API request - fetch the table
cycle_hire_table = client.get_table(cycle_hire_table_ref)
cycle_stations_table = client.get_table(cycle_stations_table_ref)

Using Kaggle's public dataset BigQuery integration.


In [21]:
# Preview the first five lines of the cycle_hire_table
client.list_rows(cycle_hire_table, max_results=5).to_dataframe() 

,rental_id,duration,duration_ms,bike_id,bike_model,end_date,end_station_id,end_station_name,start_date,start_station_id,start_station_name,end_station_logical_terminal,start_station_logical_terminal,end_station_priority_id
0,57870195,3840,3840000,4229,,2016-08-31 20:49:00+00:00,<NA>,"Golden Square, Soho",2016-08-31 19:45:00+00:00,<NA>,"Albert Gate, Hyde Park",3451,1132,0
1,57852555,3840,3840000,242,,2016-08-31 15:19:00+00:00,<NA>,"Embankment (Savoy), Strand",2016-08-31 14:15:00+00:00,<NA>,"Albert Gate, Hyde Park",1192,1132,0
2,57872531,3840,3840000,728,,2016-08-31 22:12:00+00:00,<NA>,"Green Park Station, Mayfair",2016-08-31 21:08:00+00:00,<NA>,"Chepstow Villas, Notting Hill",200096,1120,1
3,57995603,2820,2820000,4375,,2016-09-04 17:10:00+00:00,<NA>,"Park Lane , Hyde Park",2016-09-04 16:23:00+00:00,<NA>,"Hyde Park Corner, Hyde Park",1037,1075,0
4,57933206,2820,2820000,2992,,2016-09-02 14:19:00+00:00,<NA>,"Wellington Arch, Hyde Park",2016-09-02 13:32:00+00:00,<NA>,"Serpentine Car Park, Hyde Park",1109,1217,1


In [22]:
# Preview the first five lines of cycle_stations_table
client.list_rows(cycle_stations_table, max_results=10
        ).to_dataframe()

,id,installed,latitude,locked,longitude,name,bikes_count,docks_count,nbEmptyDocks,temporary,terminal_name,install_date,removal_date
0,126,True,51.517821,false,-0.096497,"Museum of London, Barbican",0,52,47,False,1043,2010-07-15,NaT
1,215,True,51.519069,false,-0.088285,"Moorfields, Moorgate",0,54,54,False,1092,2010-07-19,NaT
2,266,True,51.501027,false,-0.180246,"Queen's Gate (North), Kensington",0,41,40,False,1204,2010-07-21,NaT
3,313,True,51.517344,false,-0.138073,"Wells Street, Fitzrovia",0,38,36,False,3495,2010-07-23,NaT
4,427,True,51.513971,false,-0.092940,"Cheapside, Bank",0,43,43,False,22180,2011-07-15,NaT
5,492,True,51.525501,false,-0.032267,"Maplin Street, Mile End",0,39,38,False,200230,2012-01-22,NaT
6,175,True,51.521668,false,-0.079609,"Worship Street, Shoreditch",1,51,47,False,997,2018-06-22,NaT
7,378,True,51.495593,false,-0.179078,"Natural History Museum, South Kensington",1,42,39,False,2677,2011-01-27,NaT
8,98,True,51.525542,false,-0.138231,"Hampstead Road, Euston",2,54,46,False,972,2010-07-14,NaT
9,246,True,51.522853,false,-0.099994,"Berry Street, Clerkenwell",2,41,38,False,1159,2010-07-20,NaT


# EXTRACTING TABULAR DATA THROUGH BIGQUERY/SQL

TRIP DURATION

In [23]:
# Query to count the trip duration for the top 100 trips 
bike_trips_query = """
                    SELECT
                        bike_id,
                        trip_duration_minutes,
                        start_station_name,
                        end_station_name,
                        trip_year
                    FROM(
                        SELECT 
                        bike_id,
                        start_station_name,
                        end_station_name,
                       
                        DATETIME_DIFF(end_date,start_date,minute) AS trip_duration_minutes,
                      
                        EXTRACT(YEAR FROM start_date) as trip_year
                        FROM `bigquery-public-data.london_bicycles.cycle_hire`
                      
                        WHERE EXTRACT(YEAR FROM start_date) BETWEEN 2015 AND 2017
                        LIMIT 100 
                    ) AS t
                    ORDER BY trip_duration_minutes DESC
                  """

# Run the query, and return a pandas DataFrame
trip_duration_result = client.query(bike_trips_query).result().to_dataframe()
trip_duration_result.head()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bike_id,trip_duration_minutes,start_station_name,end_station_name,trip_year
0,12509,158,"Granby Street, Shoreditch","Granby Street, Shoreditch",2016
1,10624,111,"Waterloo Station 1, Waterloo","Concert Hall Approach 1, South Bank",2016
2,2313,94,"Lancaster Gate , Bayswater","Park Lane, Mayfair",2016
3,12572,47,"Gunmakers Lane, Old Ford","Gunmakers Lane, Old Ford",2016
4,9960,47,"Ranelagh Gardens, Fulham","Crawford Street, Marylebone",2016


We now have a table of the 100 longest bike trips along with their duration, origin, and destination between 2015 and 2017.


# DAILY NUMBER OF TRIPS

In [24]:
# Query to count the daily number of trips in 2015, as well as the cumulative number of trips
bike_trips_query = """
            
                    WITH trips_by_day AS
                    (
                    SELECT 
                        DATE(start_date) AS trip_date,
                        COUNT(*) as num_trips
                    FROM `bigquery-public-data.london_bicycles.cycle_hire`
                    WHERE EXTRACT(YEAR FROM start_date) = 2015
                    GROUP BY trip_date
                    ) 
                    SELECT *,
                    SUM(num_trips) 
                    OVER (
                            ORDER BY trip_date
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                         ) AS cumulative_trips
                    FROM trips_by_day
                  """

# Run the query, and return a pandas DataFrame
number_of_trips_result = client.query(bike_trips_query).result().to_dataframe()
number_of_trips_result.head() 

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,num_trips,cumulative_trips
0,2015-09-26,32635,7474415
1,2015-01-25,15094,425963
2,2015-05-24,30018,3310928
3,2015-09-08,32781,6942164
4,2015-03-15,10662,1437703


We now have a table with the number of trips on a given date in 2015 as well as the total number of tips up to that date.
## Bike Location on a Specific Day

In [25]:
# Query to identify each bikes location on a certain day at various times (4th of July, 2015)
bike_trips_query = """
                    SELECT 
                        bike_id,
                        TIME(start_date) AS trip_time,
                        -- First station id tells us which station the bike started trips at that day -- 
                        FIRST_VALUE(start_station_id)
                            OVER (
                                PARTITION BY bike_id
                                ORDER BY start_date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                             ) AS first_station_id,
                         -- Last station id tells us which station the bike ended trips at that day-- 
                         LAST_VALUE(end_station_id)
                            OVER (
                                PARTITION BY bike_id
                                ORDER BY start_date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                            ) AS last_station_id,
                         start_station_id,
                         end_station_id
                    FROM `bigquery-public-data.london_bicycles.cycle_hire`
                    -- The 21st of June is the summer solstice so which is the longest day of the year --
                    WHERE DATE(start_date) = '2015-06-21' 
                  """

# Run the query, and return a pandas DataFrame
bike_location_result = client.query(bike_trips_query).result().to_dataframe()
bike_location_result.head(10)

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bike_id,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,314,02:22:00,611,739,611,47
1,314,11:35:00,611,739,47,405
2,314,12:27:00,611,739,405,296
3,314,12:58:00,611,739,296,191
4,314,13:29:00,611,739,191,191
5,314,14:32:00,611,739,191,133
6,314,15:50:00,611,739,133,589
7,314,16:14:00,611,739,589,739
8,314,22:13:00,611,739,739,660
9,314,22:59:00,611,739,660,739


We now have a table with information on each of the movements of the bike through a certain day, which could help us identify the location of a certain at various times 

# BIKE STATIONS: TOTAL NUMBER OF TRIPS AND AVERAGE TRIP DURATION

In [26]:
# Query to count the number of trips by start station
join_query = """
            
             WITH c AS
             (
             SELECT 
                 start_station_id, 
                  
                 COUNT(*) as number_of_trips,
                 
                 ROUND(AVG(trip_duration_minutes)) AS avg_trip_duration_minutes
             FROM (
                 SELECT
                 start_station_id,
               
                 DATETIME_DIFF(end_date,start_date,minute) AS trip_duration_minutes,
                 FROM `bigquery-public-data.london_bicycles.cycle_hire`
             ) 
             GROUP BY start_station_id
             )
             SELECT 
                 s. id as station_id, 
                 s.name,
                 c.number_of_trips,
                 c.avg_trip_duration_minutes 
             FROM `bigquery-public-data.london_bicycles.cycle_stations` AS s
             LEFT JOIN c
             ON s.id = c.start_station_id
             ORDER BY c.number_of_trips DESC
             """

# Run the query, and return a pandas DataFrame
station_trip_duration_result = client.query(join_query).result().to_dataframe()
station_trip_duration_result.head()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,station_id,name,number_of_trips,avg_trip_duration_minutes
0,191,"Hyde Park Corner, Hyde Park",658129,41.0
1,14,"Argyle Street, Kings Cross",579703,17.0
2,154,"Waterloo Station 3, Waterloo",512990,15.0
3,303,"Albert Gate, Hyde Park",451863,37.0
4,307,"Black Lion Gate, Kensington Gardens",451696,44.0


Now we have a  table with statistics on the number of trips and average number of trips starting at the station 

# DATA ANALYSIS

Now that we have completed extracting the information we can perform some basic analysis on the data.

In [28]:
trip_duration_result.groupby('trip_year').agg(
    total_trip_duration=('trip_duration_minutes', 'sum'),
    avg_trip_duration=('trip_duration_minutes', 'mean')
)

,total_trip_duration,avg_trip_duration
trip_year,,
2016,2015,20.15


2016 was the year when most ride and average trip duration took place. 

# BIKE LOCATION AND NUMBER OF TRIPS

In [29]:
#Identifying the top five stations where trips began
bike_location_result['end_station_id'].value_counts().head(5)

end_station_id
191    726
307    439
303    389
248    337
404    328
Name: count, dtype: Int64

In [30]:
#Identifying the top five stations where trips began
bike_location_result['start_station_id'].value_counts().head(5)

start_station_id
191    739
307    436
303    400
248    328
404    316
Name: count, dtype: Int64

# BIKE STATIONS

In [31]:
#Identifying station with the longest averge trip duration.
station_trip_duration_result[station_trip_duration_result['avg_trip_duration_minutes'] == station_trip_duration_result
['avg_trip_duration_minutes'].max()]

,station_id,name,number_of_trips,avg_trip_duration_minutes
14,785,"Aquatic Centre, Queen Elizabeth Olympic Park",298695,46.0


In [32]:
#Identifying station with the least number of trips
station_trip_duration_result[station_trip_duration_result['number_of_trips'] == station_trip_duration_result['number_of_trips'].min()]

,station_id,name,number_of_trips,avg_trip_duration_minutes
784,850,"Brandon Street, Walworth",276,16.0
